In [5]:
import math
import numpy as np
import pylab
import scipy.fftpack
import scipy.interpolate
import scipy.io
import h5py

In [24]:
import numpy as np

x = np.random.random(10)
y = np.random.random(10)
print('x: ', x)
print('y: ', y)

x:  [ 0.14159234  0.32286664  0.67314182  0.10510791  0.78486943  0.30409225
  0.62241936  0.57303836  0.76982663  0.00533202]
y:  [ 0.38812464  0.27279585  0.53775224  0.58767443  0.17770843  0.16680055
  0.10315586  0.60169223  0.54426243  0.74569236]


In [30]:
data = np.array(list(zip(x, y)), dtype={'names':['col1', 'col2'], 'formats':['float','float']})
print(data['col1'])
print(data['col2'])

[ 0.14159234  0.32286664  0.67314182  0.10510791  0.78486943  0.30409225
  0.62241936  0.57303836  0.76982663  0.00533202]
[ 0.38812464  0.27279585  0.53775224  0.58767443  0.17770843  0.16680055
  0.10315586  0.60169223  0.54426243  0.74569236]
